### 4.1.1 에이다부스트 소개
새로운 트리가 이전 트리의 오차를 기반으로 가중치를 조정  
오류 샘플의 가중치를 높여 잘못도니 예측에 더 많은 주의를 기울인다.  
  
일반적으로 부스팅은 강력한 기반 모델을 만드는 것이 아니라 반복적으로 오류를 고치는 데 초점을 맞춘다.  
기반 모델이 너무 강력하면 학습 과정이 제한되어 부스팅 모델의 전략을 약화시킨다.  
  
가장 중요한 에이다부스트 매개변수는 강력한 학습기를 만들기 위해 필요한 트리 개수(반복 횟수)인 n_estimator이다. 

### 4.1.2 그레이디언트 부스팅의 특징
에이다부스트와 다른 전략을 사용한다.  
그레이디언트 부스팅도 잘못된 예측을 기반으로 조정되지만 한 단계 더 나아가 이전 트리의 예측 오차를 기반으로 완전히 새로운 트리를 훈련한다.  

오차에만 초점을 맞추는 머신러닝 알고리즘을 만들려면 정확한 최종 예측을 만들기 위해 오차를 계산하는 방법이 필요하다.  
이런 방법은 예측과 실제 값 사이의 차이인 잔차(residual)를 활용한다.  

## 4.2 그레이디언트 부스팅 작동 방식
### 4.2.1 잔차
잔차는 타깃과 모델의 예측 사이의 차이  
ex1) 자전거 대여  
 a) 예측: 759  
 b) 타깃: 799  
 c) 잔차: 799 - 759 = 40  
  
ex2) 소득  
 a) 예측: 100,000  
 b) 타깃: 88,000  
 c) 잔차: 88,000 - 100,000 = -12,000 

### 4.2.2 그레이디언트 부스팅 모델 구축 방법 배우기

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
xgb.set_config(verbosity=0)

In [3]:
df_bikes = pd.read_csv('handson-gb-main/Chapter02/bike_rentals_cleaned.csv')
df_bikes.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1.0,0.0,1.0,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1.0,0.0,1.0,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,1.0,0.0,1.0,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,1.0,0.0,1.0,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,1.0,0.0,1.0,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,1600


In [4]:
X_bikes = df_bikes.iloc[:, :-1]
y_bikes = df_bikes.iloc[:, -1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

In [5]:
from sklearn.tree import DecisionTreeRegressor 
tree_1 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_1.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [6]:
y_train_pred = tree_1.predict(X_train)

In [7]:
y2_train = y_train - y_train_pred

trainset에 대한 잔차를 구한다.  
잔차는 다음 트리의 타깃이 되기 때문에 y2_train이라고 이름을 지었다.  
  
새로운 트리를 이 잔차에 대해 훈련한다.  
이 과정이 반복되면 잔차는 0에 가까워진다.  

In [8]:
tree_2 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_2.fit(X_train, y2_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [9]:
y2_train_pred = tree_2.predict(X_train)
y3_train = y2_train - y2_train_pred
tree_3 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_3.fit(X_train, y3_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [10]:
y1_pred = tree_1.predict(X_test)
y2_pred = tree_2.predict(X_test)
y3_pred = tree_3.predict(X_test)

y_pred = y1_pred + y2_pred + y3_pred

테스트 세트에 대해 각 트리의 예측을 만들어 모두 더한다. (첫번째 예측과 잔차 예측값을 모두 더함)

In [11]:
from sklearn.metrics import mean_squared_error as MSE 
MSE(y_test, y_pred) ** 0.5

911.0479538776444

## 4.4 빅 데이터 다루기 - 그레이디언트 부스팅 vs XGBoost 
### 4.4.2 외계 행성 데이터셋 전처리

In [13]:
df = pd.read_csv('handson-gb-main/Chapter04/exoplanets.csv')
df.head()

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,2,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,2,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,2,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,2,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5087 entries, 0 to 5086
Columns: 3198 entries, LABEL to FLUX.3197
dtypes: float64(3197), int64(1)
memory usage: 124.1 MB


In [15]:
df.isnull().sum().sum()

0

In [16]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

### 4.4.3 그레이디언트 부스팅 분류 모델 만들기

In [17]:
from sklearn.ensemble import GradientBoostingClassifier 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score 

### 4.4.4 시간 측정

In [19]:
import time
start = time.time()
df.info()
end = time.time()
elapsed = end - start 
print('\n실행 시간: ' + str(elapsed) + ' 초')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5087 entries, 0 to 5086
Columns: 3198 entries, LABEL to FLUX.3197
dtypes: float64(3197), int64(1)
memory usage: 124.1 MB

실행 시간: 0.0139923095703125 초


### 4.4.5 속도 비교
GradientBoostingClassifier와 XGBoostClassifier의 속도 비교 

In [20]:
start = time.time()
gbr = GradientBoostingClassifier(n_estimators=100, max_depth=2, random_state=2)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('점수: ' + str(score))
end = time.time()
elapsed = end - start 
print('실행 시간: ' + str(elapsed) + ' 초')

점수: 0.9874213836477987
실행 시간: 157.3333990573883 초


In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [24]:
start = time.time()
xg_reg = XGBClassifier(n_estimators=100, max_depth=2)
xg_reg.fit(X_train, y_train)
y_pred = xg_reg.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('점수: ' + str(score))
end = time.time()
elapsed = end - start 
print('실행 시간: ' + str(elapsed) + ' 초')

점수: 0.0
실행 시간: 6.485869884490967 초
